In [1]:
import scipy.io as io
import plotly.offline as py
import plotly.graph_objs as go
import json
import numpy as np

py.init_notebook_mode(connected=True)

In [2]:
mat = io.loadmat('data_Mg_GBperatom_seg_2Al_dump.mat')

length_A = mat['A'].shape[1]

In [3]:
for i in range(30):
    segE = mat['A']['Eseg'][0,i]
    #check whether this is a valid data?
    n1 = segE[:,0] != 0 
    segE = np.squeeze(segE[n1,:])
    atom_ID = segE[:,0].astype(int) - 1

    descriptor = mat['A']['peratom'][0,i][0,0]
    descriptor_temp = np.concatenate([descriptor['pos'],descriptor['pe'],descriptor['cna'],descriptor['centro_fnn'],
                                descriptor['centro_snn'],descriptor['coord'],descriptor['f'],descriptor['stress'],
                                descriptor['voronoi']], axis = 1)
    if i == 0:
        descriptor_all = descriptor_temp[atom_ID]
        segE_all = segE
    else:
        descriptor_temp = descriptor_temp[atom_ID]
        descriptor_all = np.concatenate([descriptor_all, descriptor_temp], axis = 0)
        segE_all = np.concatenate([segE_all, segE])

descriptor_all[:,2] = abs(descriptor_all[:,2]-min(descriptor_all[:,2])-20)
sigma_H = np.sum(descriptor_all[:,11:14], axis = 1)/3
f_mag = np.linalg.norm(descriptor_all[:,8:11], axis = 1, ord = 2)

feature = np.concatenate([descriptor_all, sigma_H[:,np.newaxis], f_mag[:,np.newaxis]], axis = 1)

In [4]:
y_true = segE_all[:,1]
#We should consider the coordinates don't matter.
feature1 = feature[:,3:]
feature1 = feature1/np.max(feature1)
feature2 = np.zeros([feature1.shape[0], 18*18])
for i in range(18):
    feature2[:,i*18:i*18+18] = feature1[:,i][:,np.newaxis]*feature1[:,:]
feature_space = np.concatenate([feature1, feature2], axis = 1)
feature_space = np.concatenate([np.ones((feature1.shape[0],1)), feature_space], axis = 1)
np.random.seed(10)
idx0 = np.random.permutation(np.arange(len(feature_space)))
feature_space = feature_space[idx0]
y_true = y_true[idx0]
#divide data into training, crossvalildation and test sets.
X_train = feature_space[0:int(len(feature_space)*.7)]
#mean = np.mean(X_train, axis = 0)
#std = np.std(X_train, axis = 0)
#X_train = (X_train - mean)/std
X_temp = feature_space[int(len(feature_space)*.7):]
#X_temp = (X_temp - mean)/std
X_cos = X_temp[:int(len(X_temp)*.5)]
X_test = X_temp[int(len(X_temp)*.5):]
Y_train = y_true[0:int(len(feature_space)*.7)]
Y_temp = y_true[int(len(feature_space)*.7):]
Y_cos = Y_temp[:int(len(X_temp)*.5)]
Y_test = Y_temp[int(len(X_temp)*.5):]

In [5]:
w_pre=np.linalg.pinv(X_train).dot(Y_train)
y1_pre = X_train.dot(w_pre)
np.power(np.linalg.norm(y1_pre-Y_train), 2)

1.244400605905947

In [6]:
Y_pre_test = X_test.dot(w_pre)
Y_mean = np.mean(Y_test)
SS_tot = np.sum(np.power(Y_test - Y_mean, 2))
SS_res = np.power(np.linalg.norm(Y_pre_test-Y_test), 2)
R_squared = 1-SS_res/SS_tot

In [7]:
y1_pre - Y_train

array([ 0.00891225,  0.00139839,  0.00079605, ...,  0.00051774,
       -0.03448741,  0.00200688])

In [8]:
w_pre1 = np.linalg.pinv(X_train).dot(y1_pre - Y_train)
y2_pre = X_train.dot(w_pre1)

In [9]:
np.power(np.linalg.norm(y1_pre + 0.01*y2_pre - Y_train), 2)

1.2444117169613924